In [13]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    tool = TavilySearchResults(max_results=2)
    tools = [tool]

    graph_builder = StateGraph(State)
    llm = ChatOpenAI(model_name=MODEL_NAME)
    llm_with_tools = llm.bind_tools(tools)

    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}

    graph_builder.add_node("chatbot", chatbot)
    tool_node = ToolNode(tools)
    graph_builder.add_node("tools", tool_node)
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    graph_builder.set_entry_point("chatbot")
    memory = MemorySaver()
    graph = graph_builder.compile(checkpointer=memory)

    return graph

# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    for event in events:
        print(event["messages"][-1].content, flush=True)

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
graph = build_graph(MODEL_NAME)

# メインループ
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    stream_graph_updates(graph, user_input)

こんにちは！
こんにちは！何かお手伝いできることはありますか？
1たす2は？
1たす2は3です。何か他に質問がありますか？
台湾観光について検索結果を教えて

[{"url": "https://www.jstage.jst.go.jp/article/jitr/10/2/10_KJ00009976605/_pdf/-char/ja", "content": "は、こ の時期を観 光草創期とする。 概略史からの時代区分 太平洋戦争後、 台湾に おける観光事業は、 台湾省が 日本政府か ら施設や行政制度を接収して、そ のままを 経営を行っ た。 1949 年大陸本土に、 中共政権が誕生し、 同年中華民国の中央政府は南京か ら台北に移された。 1953 年台湾の経済計画の一環として、 観光事業が計画 された。1956 年の来台（ 台湾） へ の観光客は 14， 974 人（ 図 1） 、 観光外貨収入約＄936 ． OGO ドル である。 ただし、 1958 年には金門で中国と砲撃戦が起こり、国際間で は台湾 が まだ戦争状磐であるとの不安か ら、台湾に来 る観光 客の数が初めて 8 ％の マ イナス成長とな っ た｛ 張朝服、 1990 ） 6 ，。 観光外貨の収入 も減少 した。しか し、 そ の後、 社会が安定し、 観光産業も進んで い った。 〔 37 1956 60 65 ro 75 ＆ 0 85 9092 9597 （ 年｝ （ 資料：観光局発行1956− ・ 19 年の統計年報か ら作成） 図1　台湾入海外旅行者数・ 訪台湾外国人の推移 旅行業 台湾光復 （ 注1 ＞後、19S2 ． 年までに、中国、 欧亜、 遠東旅行社が設立された s ｝。1958 年、 オランダの国際 観光船 「ス タ ン テ ン」 号の乗客 3， 878 人とアメリカの軍 艦の兵士及び家族1 万人あ ／ t まりの斡旋のために． 亜美、 亜洲、 欧美旅行会社が設立さ れ た 5 〕。196e 年 に来台の 観光客は23 ， 639 人で、旅行業者は8 社である s ｝。この 8 社 の経営形態と職員の役割につ いては以下 のような 特徴がある。まず第一に8 社の社員はすべ て公務員で あ っ たこ と。第二 に 1960 年から旅行業者は民営で [...] 旅行業者 が増 加す る 